In [1]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage

# Install the packages
! pip install google-cloud-vision



INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.1
    Uninstalling google-api-core-1.34.1:
      Successfully uninstalled google-api-core-1.34.1
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.14.0
    Uninstalling google-cloud-storage-2.14.0:
      Successfully uninstalled google-cloud-storage-2.14.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.58.0
    Uninstalling google-cloud-aiplatform-1.58.0:
      Successfully uninstalled 

In [3]:
!pip install --upgrade google-cloud-videointelligence

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: google-cloud-videointelligence
    Found existing installation: google-cloud-videointelligence 1.16.3
    Uninstalling google-cloud-videointelligence-1.16.3:
      Successfully uninstalled google-cloud-videointelligence-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
google-api-python-client 1

In [62]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"


In [2]:
import json
import os
image_directory = "SampleImage"

image_names=[]
for file_name in os.listdir(image_directory):
    if  not file_name.startswith('.'):
        image_names.append(file_name)

image_paths = [os.path.join(image_directory, image_name) for image_name in image_names]


### Define function to detect explicit images

enable clound vision api before running this bit

In [11]:
# from google.cloud.videointelligence.v1beta import SafeSearchAnnotation


In [3]:
#commented as we do not need this for now 

from typing import Optional

from google.cloud import vision
from google.cloud.vision_v1.types.image_annotator import SafeSearchAnnotation

client = vision.ImageAnnotatorClient()


def detect_safe_search(path: str) -> Optional[SafeSearchAnnotation]:
    """Detects unsafe features in the file."""

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)

    if response.error.message:
        print(response.error.message)
        return None

    return response.safe_search_annotation

In [4]:
from google.cloud.vision_v1.types.image_annotator import Likelihood


# Returns true if some annotations have a potential safety issues
def convert_annotation_to_safety(safe_search_annotation: SafeSearchAnnotation) -> bool:
    return all(
        [
            (safe_level == Likelihood.VERY_UNLIKELY)
            or (safe_level == Likelihood.UNLIKELY)
            for safe_level in [
                safe_search_annotation.adult,
                safe_search_annotation.medical,
                safe_search_annotation.violence,
                safe_search_annotation.racy,
            ]
        ]
    )

In [43]:
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Optional

import numpy as np
from tqdm import tqdm

# Create a rate limiter with a limit of 1800 requests per minute
seconds_per_job = 1 / (1800 / 60)


def process_image(image_path: str) -> Optional[bool]:
    try:
        annotation = detect_safe_search(image_path)

        if annotation:
            return convert_annotation_to_safety(safe_search_annotation=annotation)
        else:
            return None
    except Exception:
        return None


# Process images using ThreadPool
is_safe_values_cloud_vision = []
with ThreadPoolExecutor() as executor:
    futures = []
    for img_url in tqdm(image_paths, total=len(image_paths), position=0):       
        #futures.append(executor.submit(process_image, img_url))
        time.sleep(seconds_per_job)

    for future in futures:
        is_safe_values_cloud_vision.append(future.result())

# Set Nones to False
is_safe_values_cloud_vision = [
    is_safe or False for is_safe in is_safe_values_cloud_vision
]

# Print number of safe images found
print(
    f"Safe images = {np.array(is_safe_values_cloud_vision).sum()} out of {len(is_safe_values_cloud_vision)} images"
)

100%|██████████| 10/10 [00:00<00:00, 29.52it/s]

Safe images = 0.0 out of 0 images


In [5]:
#for now the api is not active just to save costs. I set everything to true
is_safe_values_cloud_vision=[True for i in range (len(image_paths))]
is_safe_values_cloud_vision

[True, True, True, True, True, True, True, True, True, True]

In [6]:
# Filter images by safety
image_names = [
    image_name
    for image_name, is_safe in zip(image_names, is_safe_values_cloud_vision)
    if is_safe
]
image_paths = [
    image_path
    for image_path, is_safe in zip(image_paths, is_safe_values_cloud_vision)
    if is_safe
]

### Defining encoding functions
Create an EmbeddingPredictionClient which encapsulates the logic to call the embedding API.

In [23]:
import base64
import time
import typing

from google.cloud import aiplatform
from google.protobuf import struct_pb2

#libraries to generate image summaries
from vertexai.vision_models import MultiModalEmbeddingModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.vision_models import MultiModalEmbeddingModel
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part,
    HarmBlockThreshold,
    HarmCategory,
)

text_embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")
#multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained(
 #   "multimodalembedding@001"
#)


class EmbeddingResponse(typing.NamedTuple):
    text_embedding: typing.Sequence[float]
    image_embedding: typing.Sequence[float]


def load_image_bytes(image_uri: str) -> bytes:
    """Load image bytes from a remote or local URI."""
    image_bytes = None
    if image_uri.startswith("http://") or image_uri.startswith("https://"):
        response = requests.get(image_uri, stream=True)
        if response.status_code == 200:
            image_bytes = response.content
    else:
        image_bytes = open(image_uri, "rb").read()
    return image_bytes


class EmbeddingPredictionClient:
    """Wrapper around Prediction Service Client."""

    def __init__(
        self,
        project: str,
        location: str = "us-central1",
        api_regional_endpoint: str = "us-central1-aiplatform.googleapis.com",
    ):
        client_options = {"api_endpoint": api_regional_endpoint}
        # Initialize client that will be used to create and send requests.
        # This client only needs to be created once, and can be reused for multiple requests.
        self.client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )
        self.location = location
        self.project = project

    def get_embedding7_(self, text: str = None, image_file: str = None):
        if not text and not image_file:
            raise ValueError("At least one of text or image_file must be specified.")

        # Load image file
        image_bytes = None
        if image_file:
            image_bytes = load_image_bytes(image_file)

        instance = struct_pb2.Struct()
        if text:
            instance.fields["text"].string_value = text

        if image_bytes:
            encoded_content = base64.b64encode(image_bytes).decode("utf-8")
            image_struct = instance.fields["image"].struct_value
            image_struct.fields["bytesBase64Encoded"].string_value = encoded_content

        instances = [instance]
        endpoint = (
           f"projects/{self.project}/locations/{self.location}"
           "/publishers/google/models/multimodalembedding@001"
        )
        #response = self.client.predict(endpoint=endpoint, instances=instances)

        text_embedding=None
        image_embedding = None
        print(image_file)
        if 1==1:
            image_emb_value = [1,2,3]
            image_embedding = [v for v in image_emb_value]

        return EmbeddingResponse(
            text_embedding=text_embedding, image_embedding=image_embedding
        )
    
    def get_embedding(self, text: str = None, image_file: str = None):
        if not text and not image_file:
            raise ValueError("At least one of text or image_file must be specified.")

        # Load image file
        image_bytes = None
        if image_file:
            image_bytes = load_image_bytes(image_file)

        instance = struct_pb2.Struct()
        if text:
            instance.fields["text"].string_value = text

        if image_bytes:
            encoded_content = base64.b64encode(image_bytes).decode("utf-8")
            image_struct = instance.fields["image"].struct_value
            image_struct.fields["bytesBase64Encoded"].string_value = encoded_content

        instances = [instance]
        endpoint = (
           f"projects/{self.project}/locations/{self.location}"
           "/publishers/google/models/multimodalembedding@001"
        )
        response = self.client.predict(endpoint=endpoint, instances=instances)

        text_embedding = None
        if text:
            text_emb_value = response.predictions[0]["textEmbedding"]
            text_embedding = [v for v in text_emb_value]

        image_embedding = None
        if image_bytes:
            image_emb_value = response.predictions[0]["imageEmbedding"]
            image_embedding = [v for v in image_emb_value]

        return EmbeddingResponse(
            text_embedding=text_embedding, image_embedding=image_embedding
        )
    def get_image_summarycontent_(self, text: str = None, image_file: str = None):
        
 
        print('summary content extraction'+image_file)
        response=' The image shows three people: Joe Biden, a young girl, and Hunter Biden. Joe Biden is smiling and wearing a dark suit. The young girl is smiling and wearing a white dress. Hunter Biden is smiling and wearing a dark suit. The background is a photo of the White House.'
        
        
        return response

    def get_image_summarycontent(self, text: str = None, image_file: str = None):
        
        generative_multimodal_model= GenerativeModel("gemini-pro-vision")
        
        image_description_prompt="""You are an assistant tasked with summarizing images for retrieval. \
        These summaries will be embedded and used to retrieve the raw image. \
        Give a concise summary of the image that is well optimized for retrieval."""
        
        generation_config= GenerationConfig(temperature=0.2, max_output_tokens=2048) 
        
        safety_settings=  {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
        stream=True
        
        # Load the saved image as a Gemini Image Object
        image_for_gemini= Image.load_from_file(image_file)
        
        model_input=[image_description_prompt, image_for_gemini]
        
        response = generative_multimodal_model.generate_content(
        model_input,
        generation_config=generation_config,
        stream=stream,
        safety_settings=safety_settings, )
        
        
        response_list = []

        for chunk in response:
            try:
                response_list.append(chunk.text)
            except Exception as e:
                print(
                    "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                    e,
                )
                response_list.append("Exception occurred")
                continue
        response = "".join(response_list)
 
        return response

    def get_summarycontent_embedding_from_text_embedding_model_(self, text: str, return_array: Optional[bool] = False,) -> list:
        """
        Generates a numerical text embedding from a provided text input using a text embedding model.

        Args:
            text: The input text string to be embedded.
            return_array: If True, returns the embedding as a NumPy array.
                          If False, returns the embedding as a list. (Default: False)

        Returns:
            list or numpy.ndarray: A 768-dimensional vector representation of the input text.
                                   The format (list or NumPy array) depends on the
                                   value of the 'return_array' parameter.
        """

        #the given text is maximum 2048 token. If more, it has to be chunked.
        #embeddings = text_embedding_model.get_embeddings([text])
        #text_embedding = [embedding.values for embedding in embeddings][0]
        text_embedding=[1,2,3,4]
        print('summary content embedding----')
        print(text)
        if return_array:
            text_embedding = np.fromiter(text_embedding, dtype=float)

        # returns 768 dimensional array
        return EmbeddingResponse(
            text_embedding=text_embedding, image_embedding=None
        )

 
    def get_summarycontent_embedding_from_text_embedding_model(self, text: str, return_array: Optional[bool] = False,) -> list:
        """
        Generates a numerical text embedding from a provided text input using a text embedding model.

        Args:
            text: The input text string to be embedded.
            return_array: If True, returns the embedding as a NumPy array.
                          If False, returns the embedding as a list. (Default: False)

        Returns:
            list or numpy.ndarray: A 768-dimensional vector representation of the input text.
                                   The format (list or NumPy array) depends on the
                                   value of the 'return_array' parameter.
        """

        #the given text is maximum 2048 token. If more, it has to be chunked.
        embeddings = text_embedding_model.get_embeddings([text])
        text_embedding = [embedding.values for embedding in embeddings][0]

        if return_array:
            text_embedding = np.fromiter(text_embedding, dtype=float)

        # returns 768 dimensional array
        return EmbeddingResponse(
            text_embedding=text_embedding, image_embedding=None
        )
    

### Create helper functions to process data in batches
Datasets can be large, so it's recommended to load a batch of data at a time into memory using a generator.

In [16]:
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Callable, Generator, List

from tqdm.auto import tqdm

#Number of API calls per second
API_IMAGES_PER_SECOND = 2

def generate_batches(
    inputs: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    """
    Generator function that takes a list of strings and a batch size, and yields batches of the specified size.
    """

    for i in range(0, len(inputs), batch_size):
        yield inputs[i : i + batch_size]



def encode_to_embeddings_chunked(
    process_function: Callable[[List[str]], List[Optional[List[float]]]],
    items: List[str],
    batch_size: int = 1,
) -> List[Optional[List[float]]]:
    """
    Function that encodes a list of strings into embeddings using a process function.
    It takes a list of strings and returns a list of optional lists of floats.
    The data is processed in chunks to prevent out-of-memory errors.
    """

    embeddings_list: List[Optional[List[float]]] = []

    # Prepare the batches using a generator
    batches = generate_batches(items, batch_size)

    seconds_per_job = batch_size / API_IMAGES_PER_SECOND

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(batches, total=len(items) // batch_size, position=0):
            futures.append(executor.submit(process_function, batch))
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())
    return embeddings_list

### Create functions that wrap embedding functions in try-except and retry logic
This particular embedding model can only process 1 image at a time, so inputs are validated to be equal to a length of 1.

In [19]:
import copy
from typing import List, Optional

import numpy as np
import requests
from tenacity import retry, stop_after_attempt

client = EmbeddingPredictionClient(project=PROJECT_ID)


# Use a retry handler in case of failure
@retry(reraise=True, stop=stop_after_attempt(3))
def encode_texts_to_embeddings_with_retry(text: List[str]) -> List[List[float]]:
    assert len(text) == 1

    try:
        return [client.get_embedding(text=text[0], image_file=None).text_embedding]
    except Exception:
        raise RuntimeError("Error getting embedding.")


def encode_texts_to_embeddings(text: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_texts_to_embeddings_with_retry(text=text)
    except Exception:
        return [None for _ in range(len(text))]


@retry(reraise=True, stop=stop_after_attempt(3))
def encode_images_to_embeddings_with_retry(image_uris: List[str]) -> List[List[float]]:
    assert len(image_uris) == 1

    try:
        return [
            client.get_embedding(text=None, image_file=image_uris[0]).image_embedding
        ]
    except Exception as ex:
        print(ex)
        raise RuntimeError("Error getting embedding for image.")


def encode_images_to_embeddings(image_uris: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_images_to_embeddings_with_retry(image_uris=image_uris)
    except Exception as ex:
        print(ex)
        return [None for _ in range(len(image_uris))]
    

@retry(reraise=True, stop=stop_after_attempt(3))
def encode_images_to_summarycontent_with_retry(image_uris: List[str]) -> List[List[float]]:
    assert len(image_uris) == 1

    try:
        return [
            client.get_image_summarycontent(text=None, image_file=image_uris[0])
        ]
    except Exception as ex:
        print(ex)
        raise RuntimeError("Error getting summaries.")


def encode_images_to_summarycontent(image_uris: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_images_to_summarycontent_with_retry(image_uris=image_uris)
    except Exception as ex:
        print(ex)
        return [None for _ in range(len(image_uris))]
    
    
# Use a retry handler in case of failure
@retry(reraise=True, stop=stop_after_attempt(3))
def encode_summarycontent_to_embeddings_with_retry(text: List[str]) -> List[List[float]]:
    assert len(text) == 1

    try:
        return [client.get_summarycontent_embedding_from_text_embedding_model(text=text[0]).text_embedding]
    except Exception:
        raise RuntimeError("Error getting embedding for summary content.")


def encode_summarycontent_to_embeddings(text: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_summarycontent_to_embeddings_with_retry(text=text)
    except Exception:
        return [None for _ in range(len(text))]
    

### Create and save the embeddings in JSONL format
The data must be formatted in JSONL format, which means each embedding dictionary is written as an individual JSON object on its own line.

See more information in the docs at Input data format and structure.

Run the following code in the next available cells, to create a temporary file to store embeddings in JSON format.

In [11]:
import tempfile, shutil

# Create temporary file to write embeddings to
embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

# Create temporary file to write summaries to
summaries_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)


In [18]:
image_paths[0 : 0 + 3]

['SampleImage/7944pcnf.png',
 'SampleImage/lf2aujik.png',
 'SampleImage/ka7ggrz8.png']

### embedding file


In [30]:
import json

BATCH_SIZE = 2# this can be changed

with open(embeddings_file.name, "a") as ef, open(summaries_file.name, "a") as sf:
     for i in tqdm(range(0, 2, BATCH_SIZE)):#len(image_names)
        image_names_chunk = image_names[i : i + BATCH_SIZE]
        image_paths_chunk = image_paths[i : i + BATCH_SIZE]
        embeddings=[]
        image_summaries=[]
        #comment to prevent extra costs
        #********************************
        #embeddings = encode_to_embeddings_chunked(
            #process_function=encode_images_to_embeddings, items=image_paths_chunk
        #)
        
        #********************************
        embeddings=[["-0.00444104", "0.0210341047", "-0.0173655078", "0.00955265295", "-0.0127431", "0.0304760132", "-0.00220620399", "0.0184926782", "-0.00321614509", "0.00739449169", "0.0259695556", "-0.00304896012", "-0.00780121796", "-0.0509818681", "-0.00216351566", "-0.0196360387", "-0.00188136974", "-0.0168304425", "-0.0543367341", "-0.0100780325", "-0.120107651", "-0.0243432857", "0.00445867563", "-0.0144569715", "-0.0106938435", "0.0104202135", "-0.0139829824", "0.0142488638", "-0.0521037206", "0.0044897031", "0.0282531139", "0.0185917187", "0.0254540164", "-0.0252824221", "-0.00611533597", "-0.0356582478", "-0.0060582296", "-0.000551598961", "-0.0112998197", "0.0223956686", "0.0310282502", "-0.0261098929", "0.0162973776", "0.0430411696", "0.0175687186", "0.00854843576", "0.0194278434", "-0.0293027181", "-0.024950156", "0.0035575463", "0.00582783855", "-0.0256006233", "-0.00415061414", "-0.00844062", "0.00565143395", "0.0157454368", "-0.00574005209", "0.0100544309", "0.013920323", "5.81386703e-05", "-0.0309475083", "-0.0500125811", "-0.0107541457", "0.00563458866", "0.0152888494", "0.0228430498", "-0.012616422", "-0.00577362347", "0.0576310381", "-0.00990470592", "-0.0102263913", "-0.000659283", "0.0158234537", "0.0122115407", "0.000500683265", "-0.0151799908", "-0.0506479479", "0.0221303627", "-0.0386895612", "0.020270694", "0.034621004", "0.00256243278", "-0.00891586207", "-0.0143985739", "-0.00695679383", "-0.00663886964", "-0.0304445457", "-0.0156727396", "0.00420973031", "-0.0586698838", "0.00410825387", "0.0271232426", "-0.0138427662", "-0.159365267", "0.0203828551", "-0.00161311857", "-0.00562610663", "0.0154484045", "-0.0111947572", "0.000672979804", "0.0150150592", "-0.00813692249", "0.0095739942", "0.0230081137", "0.0173290893", "-0.0692063645", "0.00598813174", "-0.0122639351", "-0.000919678598", "0.00873339735", "-0.017490387", "-0.0218969323", "-0.0138157103", "0.000122752099", "-0.0119467918", "0.00444263499", "0.0128962081", "-0.0244083591", "-0.00547026191", "-0.0390731096", "0.00984373502", "-0.0839248", "-0.00390384905", "0.0106456699", "0.00403241115", "-0.068637751", "-0.0730250254", "-0.0321484469", "0.0306083094", "0.00282459", "-0.00789664313", "0.0471494086", "0.00321116857", "0.00807952601", "0.056837976", "-0.0125541883", "-0.00767001742", "-0.0525370687", "0.0444531068", "0.0453908332", "0.000530325866", "-0.00329720532", "0.0286210794", "0.00408606138", "-0.00727501512", "0.0231464189", "-0.0146673638", "-0.0149145182", "-0.0308361147", "-0.00979910418", "0.0714274645", "0.0582080521", "-0.0268705953", "-0.00671728514", "0.0029173519", "-0.00020530523", "-0.0180394035", "0.0101269213", "0.0061199246", "-0.0148288943", "-0.0173991211", "0.00415616343", "0.00516958116", "0.0185271446", "0.00585192768", "-0.0374451503", "-0.00631989492", "0.0183783527", "-0.0194372702", "0.0149785569", "0.0142473578", "0.0393038802", "0.0222360324", "-0.0239168871", "-0.0234786142", "-0.000900907617", "0.0155002903", "-0.00933530368", "-0.0283041764", "-0.0260368418", "0.00348003954", "-0.0144895231", "0.00707295956", "-0.0046277931", "-0.0166957583", "0.0102566043", "0.0290871263", "0.00394689664", "0.0168205202", "0.0138228647", "-0.0120789511", "-0.0190399513", "0.0032271354", "-0.00788813364", "-0.0232635625", "0.0298521724", "0.018985983", "-0.0517985821", "0.00404002564", "-0.000163968434", "0.0223093182", "0.00316403643", "-0.000638289202", "-0.00129216234", "-0.0399786867", "-0.0076194955", "-0.0206234325", "-0.00318609807", "-0.0107595436", "-0.000110840927", "0.00872531", "0.00645321095", "0.013954021", "0.0561077036", "-0.0253666248", "0.0015998747", "-0.0258359015", "0.00244076131", "0.015436437", "0.00226330291", "0.00260188687", "-0.0156781524", "-0.0408380404", "-0.02431098", "-0.0114595536", "-0.0374333039", "-0.0131775066", "0.0387165807", "-0.0484433807", "-0.0595076233", "-0.0258893725", "-0.0165955331", "-0.0293637719", "-0.0227647424", "-0.0247916263", "-0.0222008266", "-0.0125031555", "0.0048615085", "-0.0212250482", "-0.0221667625", "-0.0142507153", "-0.00891591795", "-0.0327359177", "0.027559692", "-0.00985556375", "-0.0364227965", "-0.0215479843", "0.00909041706", "0.0124219507", "-0.0148399165", "0.0124513824", "0.0313181058", "-0.00200246228", "0.00182055461", "-0.023112271", "0.00671931", "-0.00653608", "-0.0113852005", "-0.00540999742", "-0.0350458771", "-0.00170907495", "0.0019060293", "-0.015964523", "-0.0187281538", "-0.0184523724", "0.0133504802", "0.025463257", "0.0310915392", "0.0373783708", "-0.00974269211", "-0.00278381305", "0.0236023646", "-0.0302019436", "0.00616766745", "-0.00102763029", "0.0353309587", "0.0241611414", "0.0034601714", "0.0213340428", "0.0976887122", "-0.046335768", "-0.0249372646", "-0.0462668836", "-0.00392812863", "0.022209093", "-0.0148670245", "-0.0623159669", "-0.0246347021", "0.00732682878", "0.0027423487", "-0.00310807675", "0.027237257", "-0.0435814485", "0.0125931334", "0.00703645451", "-0.0333511196", "-0.00346098794", "0.00719646225", "0.00315731298", "-0.020855261", "0.0101266662", "0.035382919", "0.0667850822", "0.00392793538", "-0.0297206268", "0.000597769395", "-0.0672169924", "0.0202295855", "0.0129406713", "0.0211672988", "0.00444812654", "-0.00486393459", "0.0310801826", "0.0300486367", "0.00116495299", "-0.0502911359", "0.00409852341", "-0.0241745", "0.0170698818", "0.041702643", "-0.0125867678", "-0.0387119278", "0.00794494152", "0.00108426576", "-0.0220767763", "-0.0237227", "0.0163683407", "-0.0239962284", "0.000888850482", "0.0579360947", "-0.00738380617", "0.024892984", "-0.00819080137", "-0.0177578349", "-0.0549898967", "0.00380085409", "0.0015056032", "-0.00214167195", "0.0078612119", "-0.00231911661", "0.015609568", "0.021735888", "0.00949667674", "0.0104025714", "-0.0189726558", "0.0309147909", "0.0131362686", "0.000818711938", "-0.042257268", "0.00522630103", "0.0255399924", "0.000283503177", "0.00605623191", "-0.0341302343", "0.0128808152", "-0.0132720824", "0.00899268314", "0.028336294", "-0.0255896822", "-0.0160386898", "-0.025317762", "-0.00205710949", "-0.00222434779", "0.004450846", "-0.0219843052", "-0.0417421758", "0.0384108163", "-0.0217492841", "-0.0378303714", "-0.0115889199", "-0.0164130684", "-0.0247931369", "-0.0303082354", "0.0356641822", "0.00914992392", "-0.0128256511", "0.0426416285", "-0.0531977862", "0.0193459112", "-0.000323638931", "0.0241938755", "0.00933080353", "-0.0159174427", "-0.0133922063", "-0.0302664079", "-0.00491546327", "0.0266498681", "-0.00456171483", "0.000686289801", "0.0172964483", "0.00535510667", "0.00883548148", "-0.00361371483", "-0.0345255397", "-0.0321943", "-0.0109268", "0.0108117396", "-0.0232922342", "-0.00455137854", "0.0118628051", "0.00826771557", "-0.0333965", "0.0107598454", "0.00537514267", "0.0162193906", "-0.0053449627", "-0.0253279414", "0.00518516544", "0.00563202053", "0.0128805339", "-0.0152805122", "0.031162763", "-0.00196248735", "0.00882009882", "0.0610770471", "0.0122212516", "-0.00846204534", "-0.00421236362", "0.00700583681", "-0.00852468", "0.00872922316", "0.0127292583", "-0.0210540723", "-0.00947807", "0.0381569602", "-0.0123410653", "-0.0171048082", "0.00732509932", "0.0443932936", "-0.0120589556", "-0.0162360631", "-0.0480869375", "0.000587500515", "0.0115946503", "-0.0031133804", "-0.0176426712", "0.00972736", "-0.0340848938", "0.0318872705", "-0.0201374628", "0.0473690443", "0.0118877832", "-0.0168102123", "-0.0322737843", "0.00909838732", "0.0282396879", "-0.0212043542", "0.0115684783", "-0.0322874486", "0.0173205882", "-0.00358811789", "0.0138829872", "-0.0162348878", "-0.0152331525", "0.0110520553", "-0.00225729169", "-0.0210926775", "0.00452684471", "-0.0030335749", "0.0202746782", "-0.00929534622", "-0.0171946883", "-0.0367240719", "-0.0101497527", "0.0120237721", "-0.0265934449", "-0.0158469789", "-0.0412944704", "-0.0264272504", "-0.00834009238", "-0.04090143", "-0.00736824516", "-0.0214873217", "0.0409367383", "-0.0176732875", "-0.0499754846", "0.0047317422", "0.00740995165", "0.00235663611", "0.0254160613", "0.0149849048", "-0.00554745039", "-0.0158028547", "-0.0492126122", "-0.0145012988", "-0.0258042607", "-0.0338208415", "0.0284352265", "-0.0367093571", "0.00309089781", "-0.0242375", "0.0258777235", "-0.0107649099", "-0.0416995101", "-0.00306743616", "0.0423433296", "0.0143210478", "0.0192640014", "-0.0102702379", "-1.060456e-05", "-0.024810221", "-0.00766483881", "0.0363302045", "-0.0138584794", "0.035593424", "-0.0151553983", "-0.0295873471", "-0.0109271593", "0.0154792396", "0.0236587", "0.0515781268", "0.025470769", "-0.0345232263", "0.0176570043", "-0.0042146421", "-0.0122637507", "0.0172112919", "-0.0101392139", "-0.0163399987", "-0.013688528", "-0.0249018464", "-0.0200837683", "0.0127852736", "0.00232161884", "0.0601678826", "-0.0346771032", "0.00434196321", "-0.015462744", "-0.0161179453", "-0.00235398253", "0.00420984626", "0.00256769545", "-0.0164962355", "-0.0298813768", "-0.0182086471", "0.035227403", "-0.0145190749", "0.01390584", "0.0336537249", "0.0182116982", "-0.00850074366", "-0.0211752094", "0.0193850752", "0.0389050953", "0.0239071604", "-0.0129408091", "-0.00754241599", "0.0145663545", "0.0361677408", "-0.00913643744", "0.0220086686", "-0.0412355438", "-0.00437345589", "0.0403108299", "0.0244388264", "-0.00675311126", "-0.0396538973", "0.0247366838", "0.012891598", "-0.0233162306", "-0.0409941413", "-0.0152231138", "0.00554568553", "-0.0086988071", "0.00446092524", "-0.013927361", "0.00551653421", "0.01883444", "-0.017451413", "-0.0138021391", "0.00656708097", "-0.0145955607", "-0.0247606654", "0.0331601612", "0.0107650543", "0.0463770665", "-0.0390238576", "-0.0240544416", "-0.0279449336", "0.00317539135", "-0.0065622204", "0.0110646756", "-0.0230470635", "-0.0399116725", "-0.00127961102", "-0.00356074912", "0.0434184596", "-0.012863203", "-0.00776370196", "0.018215904", "-0.0301844701", "-0.0151422089", "-0.0356385745", "-0.00629722932", "0.00505530229", "-0.0182746518", "0.00837502629", "0.0407357961", "-0.0239848606", "-0.00840649847", "0.0526366867", "-0.00695059309", "0.00885057356", "-0.0228431709", "-0.0138602722", "0.0116470102", "-0.00830822717", "-0.00806239899", "0.010166646", "-0.00248893071", "-0.00436328", "0.0155207403", "0.0438759029", "0.0454749465", "-0.00683911797", "-0.0244895145", "0.0183002669", "-0.0371374302", "0.0391417108", "-0.00964155421", "0.0132377753", "-0.00672504772", "-0.00671758223", "0.00422372203", "-0.00805099402", "-0.0307236742", "-0.00810552295", "0.0143500883", "0.0545473844", "0.00972227566", "-0.0114788702", "-0.0317154862", "0.0219885167", "-0.0260560326", "0.0121788457", "-0.0631521419", "-0.0157655496", "-0.026639672", "0.0284301601", "-0.0123591637", "-0.0112761166", "-0.00093679718", "-0.0194497295", "0.00724800723", "-0.0191989895", "0.00877077505", "0.00931578223", "0.00567167485", "0.033902254", "-0.019827012", "-0.00131691282", "-0.00543185417", "0.0141761974", "0.0288597494", "-0.00405924534", "0.0398574211", "-0.00748855388", "0.0701595098", "-0.0197905041", "0.00775466952", "-0.12345507", "-0.0224001482", "-0.00557336817", "0.00572995283", "-0.00825778581", "-0.0174930505", "0.0341090821", "-0.00330266892", "0.0589076467", "0.0246167146", "0.0210607145", "0.0267756972", "-0.235381156", "0.0663033351", "0.00827191398", "0.0203248076", "-0.00951726735", "-0.00704101752", "0.0358106159", "-0.019814847", "-0.00590557884", "0.0414790176", "0.00310882693", "0.0104895011", "0.0115022864", "-0.0216282122", "-0.0188380573", "-0.0293714404", "-0.00944028609", "0.0036465805", "0.0458181947", "-0.00352259702", "-0.0239433814", "-0.0335745476", "-0.0509180464", "0.000411464716", "0.0108110914", "-0.0421191677", "-0.012302462", "0.0266497191", "0.0280418415", "-0.0142147383", "-0.0121011389", "-0.0370630547", "-0.000397563272", "0.0285502169", "-0.0167192537", "-0.0380172916", "-3.4938279e-05", "-0.0560381562", "0.0382072031", "-0.0551106893", "-0.00688742427", "0.00952325", "0.00835159142", "0.0127986502", "0.0309025757", "-0.0365491286", "0.00611060904", "0.0173719693", "-0.0196081866", "-0.021658944", "0.0219172668", "0.0087742731", "-0.0146285063", "0.00739034358", "-0.0286736209", "0.039115563", "0.0461857878", "-0.00388035295", "-0.027320385", "0.0121737318", "-0.0122831613", "0.0128935697", "-0.0181095209", "-0.00994852744", "0.0276653338", "-0.0123509653", "-0.0103332745", "-0.0161895212", "-0.00369938649", "0.00893449225", "-0.00751358969", "0.0142603302", "0.0252360962", "0.0245206095", "-0.00300171156", "0.0116593549", "-0.238391921", "0.00666414201", "-0.0185425", "0.0544458441", "-0.0113593834", "-0.0162508264", "0.0218410119", "-0.0530925505", "0.0270753521", "0.00400130823", "-0.0269170552", "0.00422351388", "-0.00273663038", "-0.00193750346", "-0.0127555532", "0.0167655796", "-0.0286755376", "-0.0104934173", "-0.0112252822", "-0.00244689616", "0.00982025452", "-0.012015379", "-0.0262147449", "0.00504497113", "-0.0210665949", "-0.000936487", "0.0187859107", "0.0202231333", "0.0319301896", "0.0326802097", "-0.0435016192", "0.00084643095", "-0.0197617579", "-0.0156127624", "0.0147250015", "-0.00693316106", "-0.00223172246", "0.0271960329", "-0.00597509276", "0.0345396698", "0.0486691594", "0.0178971831", "0.00925191492", "0.0194945373", "-0.0170005467", "0.0199283026", "0.00118002726", "-0.00630969368", "-0.01804021", "0.0180291664", "0.00568418438", "-0.0179465935", "0.044970382", "-0.0212096311", "0.0229899921", "0.00012601359", "-0.0130985295", "0.0178971048", "0.0213922765", "-0.0143007059", "0.0533803739", "0.00554051669", "0.0129576344", "-0.00270084129", "0.0173771344", "-0.00692568952", "-0.0203674491", "-0.00733244652", "0.0134140514", "-0.0171652529", "-0.0305941161", "0.0279757809", "0.0323332138", "-0.00887311343", "0.113927774", "0.0209839288", "-0.0421417", "-0.0163171", "-0.0290548429", "-0.0546432436", "-0.009136457", "0.00659354683", "-0.00526553346", "0.019820286", "-0.0215143748", "-0.0131016234", "0.0215107892", "-0.00128397916", "0.00994382147", "0.028496312", "0.0332659259", "0.00173224939", "-0.0100251064", "-0.0306102559", "-0.0171832014", "-0.0158391986", "-0.0105313612", "0.00458520837", "0.0131103517", "-0.00560630439", "-0.0160486661", "0.0151119335", "-0.0430918671", "0.011735836", "0.00785818603", "-0.0527240336", "0.0254097413", "0.016166", "0.00957229733", "0.00245701382", "-0.0120941317", "-0.0170029048", "0.00601957319", "0.0550323278", "-0.0171307754", "0.0364138745", "-0.0272104", "0.0087683592", "0.00117934775", "-0.0209948719", "-0.0254111346", "-0.0289451871", "-0.0407819375", "-0.0282959882", "0.00283609959", "0.0250870027", "-0.0151547175", "0.0265065841", "0.018813191", "-0.00472076144", "0.0234261919", "-0.0139485402", "0.005066582", "-0.00089029863", "-0.014848371", "-0.0111990068", "0.0261144694", "0.0315814056", "0.0164719801", "0.0533070862", "0.00761417765", "-0.0456191041", "0.00551501289", "0.0110498723", "0.0282897335", "0.0101131452", "-0.00461887335", "-0.0301004", "-0.00812413171", "-0.0293664262", "-0.0127573218", "0.0127831353", "0.02002283", "-0.0355637632", "0.0204467028", "0.0317608565", "-0.0211332738", "0.0332996175", "0.0159235988", "-0.0191357248", "0.0325265825", "0.0379079022", "0.0159052294", "0.0145629542", "-0.031708315", "-0.0162033532", "-0.00158395665", "-0.0231582467", "0.0132306246", "0.00564087601", "-0.0081592584", "-0.0487954058", "-0.00117845077", "0.0125827277", "0.0096750306", "-0.00817531906", "-0.0448946953", "-0.00100910151", "-0.0079947114", "-0.0471788384", "-0.00244944054", "-0.00569055881", "0.0265971944", "-0.0102198608", "-0.000835418", "0.0102021685", "-0.0182793066", "0.0376878", "8.7153654e-05", "0.00357876113", "-0.0200664066", "-0.0256973747", "-0.00189534633", "0.0456525721", "0.0183854494", "-0.0403054841", "0.00889274105", "-0.0323965549", "-0.0341626778", "-0.0184860751", "0.00284186332", "0.0276100393", "0.0135357631", "-0.0176614672", "0.0210233293", "0.00578869088", "0.00643932866", "0.0401511863", "-0.0370330736", "-0.00337228295", "0.0229662452", "0.000221449896", "-0.0275461227", "-0.00141317875", "-0.027817212", "-0.0422420092", "0.0066121188", "0.0157616828", "0.014235097", "-0.0351489447", "-0.040109802", "-0.000118441465", "0.0216255374", "-0.0288908556", "0.00648762239", "-0.0188905858", "-0.0248837136", "-0.02190727", "0.00671791751", "0.0189760718", "-0.00388853694", "-0.0387605578", "0.013279601", "0.0178551059", "0.00627668062", "0.01817213", "-0.0110072335", "-0.00270554377", "-0.00815316", "-0.00402911799", "0.00673495093", "0.0281307492", "-0.0017002112", "0.0118889865", "-0.0479706563", "-0.0242704339", "0.00720036263", "0.0154125122", "0.0226053204", "0.00233229809", "-0.00342275086", "-0.00690834178", "0.00376485381", "0.0265539736", "-0.00572855351", "0.00468836678", "0.0181986112", "0.0173219852", "-0.0315927193", "-0.0302833319", "-0.0315297246", "0.0416783392", "-0.0353111066", "0.00691926", "0.00907775853", "-0.00638584886", "0.0125032645", "0.0254122745", "-0.00144094252", "0.0030140169", "-0.00011192316", "0.014039469", "0.00157613761", "-0.0141819092", "-0.00394541351", "-0.0298934691", "0.0216330942", "-0.0284470357", "0.0169094335", "-0.00877323467", "-0.0190338083", "-0.0292949", "-0.0205905475", "0.0264286082", "0.0129324114", "0.0252408236", "0.0306638889", "0.0170190018", "0.05162457", "-0.0240615942", "-0.0476406366", "-0.0210000277", "-0.00818918739", "-0.0274234023", "-0.00740239536", "0.0136232963", "0.0129455235", "-0.0271912571", "0.00828913", "0.015697768", "-0.0304843076", "0.00628806604", "-0.0264968202", "-0.0201471504", "0.00362884603", "-0.00386903761", "0.00168223283", "0.015565902", "-0.0224776473", "-0.0155581441", "-0.0360924713", "0.0260091387", "0.0211588107", "-0.00477207778", "-0.0513735674", "0.00770528289", "0.0587310977", "0.0124170939", "-0.0414750762", "0.0139309494", "-0.0134018818", "0.0231991615", "-0.00594919221", "0.024298979", "-0.00848027132", "-0.0382078439", "0.0172657575", "0.00683750119", "0.00632458413", "0.0440213755", "0.0085756192", "-0.0480464548", "0.0239341985", "0.0191684216", "-0.00374529324", "-0.0145738702", "0.029823225", "0.00121635257", "-0.0265159458", "0.015831735", "-0.0255386773", "0.0522183925", "-0.0305068735", "0.0276982505", "-0.0259894896", "0.0339332074", "-0.00310222525", "-0.0194109492", "-0.013573397", "-0.0478389338", "0.0272903908", "-0.020548068", "-0.0507636368", "-0.00663743541", "-0.00642128568", "-0.00951317", "0.064879261", "0.0226083267", "-0.0231766794", "-0.0135533083", "-0.0139605775", "-0.0247337595", "-0.000105913219", "0.00256505469", "-0.036242988", "-0.0164092146", "0.0234937109", "-0.015201007", "-0.013465682", "-0.0104544964", "-0.0216573607", "0.00366258691", "0.00899076089", "0.012660278", "0.0132538797", "-0.0141602801", "-0.0220480654", "0.00282695307", "-0.000456487411", "-0.00904792", "0.0440055355", "-0.0197302811", "0.00270965439", "0.0461533703", "-0.0234032404", "-0.0170412473", "0.000936214114", "-0.0310092755", "0.0262232851", "0.0203349888", "0.000786532357", "-0.0430070385", "-0.0446674936", "-0.00973727275", "-0.0133312782", "0.025027182", "-0.0169286076", "-0.02170779", "-0.00313940481", "0.00669592433", "0.023298081", "0.0373612754", "-0.00242739776", "-0.00809190143", "-0.0102204029", "-0.113423258", "0.00793535262", "-0.0231552944", "0.0172172487", "0.00224126", "-0.0261384398", "0.0106978966", "0.00247382675", "-0.0102471244", "-0.0395459123", "0.0149856117", "0.0063048983", "-0.0432128645", "0.0150740473", "0.0266760662", "0.0129045863", "-0.0313507617", "-0.0154269096", "0.0626054779", "-0.0274499338", "-0.0617214218", "0.0452942215", "-0.00325097353", "0.0204035267", "0.0101196021", "-0.0138446661", "-0.00398711674", "-0.0199280363", "0.00674943114", "-0.00232053827", "0.0129039353", "0.00160717266", "-0.0019722553", "0.00224205339", "-0.0173129514", "-0.00333447685", "0.0284881406", "-0.00156901928", "0.00480100373", "0.0255555678", "-0.00767270895", "-0.00678580953", "0.00633462472", "-0.0133158779", "0.0196514893", "0.00750389509", "-0.0344266146", "0.00777156418", "0.00660795206", "-0.0165288858", "-0.0306826811", "0.0010533724", "-0.0463804603", "-0.012116909", "0.00448443787", "-0.00717262132", "-0.00185956736", "-0.025156457", "-0.000507657649", "-0.00899541564", "-0.00476406422", "0.0148629295", "-0.0516718738", "-0.00657236809", "-0.0270420425", "-0.0159768667", "0.0148429805", "0.0122018745", "0.00221860409", "0.00525457785", "0.0247915275", "-0.0161637776", "-0.0121008018", "0.017532507", "0.00455021532", "-0.0200572778", "-0.0111056762", "0.00456012", "0.00310130464", "-0.0159439873", "-0.0339448787", "-0.00488472311", "0.0506151617", "-0.0167680029", "-0.0277899951", "-0.0557396188", "0.017725423", "-0.00582811376", "-0.0148980077", "-0.0356296599", "0.00795417652", "0.0559579954", "-0.0611429103", "0.0192459766", "-0.0426671207", "0.00196357141", "-0.0142285023", "0.00348672108", "-0.0230668522", "-0.012585436", "-0.0198518429", "-0.00885895453", "0.00318440213", "0.024423914", "-0.0219476148", "-0.0229110345", "-0.0119669372", "-0.0140682133", "0.0334332511", "-0.0148884589", "0.00165009755", "-0.00301711075", "-0.0187776852", "-0.00881295744", "0.0229435842", "0.0105057452", "-0.024525959", "0.0212991331", "-0.0150207551", "-0.0293319561", "-0.047616519", "-0.0160366967", "-0.014308257", "0.0303463507", "0.0449383073", "0.0179366581", "0.00269475277", "-0.000315558602", "0.00875211228", "-0.016219154", "0.0232554823", "0.0213728361", "-0.00686744647", "0.0377409793", "0.0426967628", "0.00391999027", "0.00826386269", "-0.0218906216", "0.0619237237", "-0.0337651707", "-0.0201235432", "-0.00201678229", "0.00117332966", "0.0094796624", "0.00132859161", "0.0192380343", "0.0141737843", "-0.0071165585", "-0.0359596126", "0.00852528308", "-0.00679441309", "0.000730822096", "0.000644947693", "-0.00580429705", "-0.0458846912", "0.0366289169", "-0.0187809151", "-0.0180011969", "0.0255250763", "-0.0146161458", "-0.0246778261", "-0.0262900423", "0.028372379", "0.0269023683", "-0.0331371836", "0.0276789777", "0.00722214114", "0.0281604901", "0.00123032264", "0.0124311987", "-0.00370868226", "0.0190311279", "-0.0985315219", "-0.0130068902", "-0.0161244739", "0.00985188503", "-0.00416125124", "-0.0390811898", "0.00136739505", "0.00170969509", "-0.0627582893", "-0.0497051775", "0.0153699452", "0.010803042", "-0.00380609976", "-0.019733794", "0.0238904171", "0.00359628885", "-0.0251013152", "-0.00233955239", "-0.00510132778", "-0.0231700558", "0.00803968403", "0.0171763767", "-0.0369831026", "-0.0187803116", "-0.00186842808", "0.0142115", "-0.0515947603", "0.0307679903", "-0.00527211372", "-0.000268350064", "0.0237011053", "0.0228533763", "0.0160283316", "0.02387443", "0.00300675514", "-0.0202850252", "-0.0279781781", "0.0280470867", "0.0473450497", "-0.000616830483", "0.0338832587", "0.0479008518", "-0.00057575776", "-0.0552125499", "-0.00757826352", "-0.0152952643", "0.0532568", "0.0116283447", "0.000412318856", "-0.0425879955", "0.0273340307", "-0.0175443478", "0.0269476213", "-0.0371628962", "0.00623885682", "-0.00968767609", "0.00865711644", "0.00817068107", "0.00534166489", "0.00816760864", "-0.0216344278", "0.0338061936", "0.0101118088", "-0.0353300311", "-0.0236998778", "0.00769372238", "0.00103167037", "-0.0227465052", "-0.032068979", "0.0428093374", "-0.0114402138", "-0.0293526947", "-0.0457064621", "-0.0273776948", "-0.0124104079", "0.0322648063", "0.031134937", "0.0114750797", "-0.0126922335", "-0.000254746294", "-0.0179353338", "-0.0282702446", "0.0417775884"],[1,2,3]]
        
        #comment to prevent extra costs
        #********************************
        #summaries = encode_to_embeddings_chunked(
            #process_function=encode_images_to_summarycontent, items=image_paths_chunk
           #)
        
        #********************************
        summaries=[' The image shows three people: Joe Biden, a young girl, and Hunter Biden. Joe Biden is smiling and wearing a dark suit. The young girl is smiling and wearing a white dress. Hunter Biden is smiling and wearing a dark suit. The background is a photo of the White House.','this is test']
   
        #comment to prevent extra costs
        #********************************
        #summaries_embeddings = encode_to_embeddings_chunked(
             #process_function=encode_summarycontent_to_embeddings, items=summaries
        #)
        summaries_embeddings=[[1,2],[1,2,3]]
        
        #********************************
        
        print(embeddings)
        print('-----')
        print(summaries)
        print('-----')
        print(summaries_embeddings)
        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(image_names_chunk, embeddings)
            if embedding is not None
        ]
        ef.writelines(embeddings_formatted)
        
        
        summaries_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "image path": image_path,
                    "summary":  summary,
                    "summary embedding": [str(value) for value in summaries_embedding],
                    "image embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, summary,summaries_embedding,embedding,image_path in zip(image_names_chunk, summaries,summaries_embeddings,embeddings,image_paths)
            if summaries is not None
        ]
        sf.writelines(summaries_formatted)
        


  0%|          | 0/1 [00:00<?, ?it/s]

[['-0.00444104', '0.0210341047', '-0.0173655078', '0.00955265295', '-0.0127431', '0.0304760132', '-0.00220620399', '0.0184926782', '-0.00321614509', '0.00739449169', '0.0259695556', '-0.00304896012', '-0.00780121796', '-0.0509818681', '-0.00216351566', '-0.0196360387', '-0.00188136974', '-0.0168304425', '-0.0543367341', '-0.0100780325', '-0.120107651', '-0.0243432857', '0.00445867563', '-0.0144569715', '-0.0106938435', '0.0104202135', '-0.0139829824', '0.0142488638', '-0.0521037206', '0.0044897031', '0.0282531139', '0.0185917187', '0.0254540164', '-0.0252824221', '-0.00611533597', '-0.0356582478', '-0.0060582296', '-0.000551598961', '-0.0112998197', '0.0223956686', '0.0310282502', '-0.0261098929', '0.0162973776', '0.0430411696', '0.0175687186', '0.00854843576', '0.0194278434', '-0.0293027181', '-0.024950156', '0.0035575463', '0.00582783855', '-0.0256006233', '-0.00415061414', '-0.00844062', '0.00565143395', '0.0157454368', '-0.00574005209', '0.0100544309', '0.013920323', '5.81386703e-0

### Create bucket and push embeddings into the bucket

In [ ]:

#set bucket info to create a bucket
BUCKET_URI = f"gs://artifacts-{PROJECT_ID}-unique"  # @param {type:"string"}
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
UNIQUE_FOLDER_NAME = "embeddings_results_NinePOC"


In [ ]:
#embeddings
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {embeddings_file.name} {EMBEDDINGS_INITIAL_URI}

#summaries
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {summaries_file.name} {EMBEDDINGS_INITIAL_URI}

In [45]:
embeddings_file.name

'/var/tmp/tmpsmy49g0o.json'

In [53]:
with open(embeddings_file.name) as f:
    for line in f:
         print(line)

{"id": "7944pcnf.png", "embedding": ["-0.00444104", "0.0210341047", "-0.0173655078", "0.00955265295", "-0.0127431", "0.0304760132", "-0.00220620399", "0.0184926782", "-0.00321614509", "0.00739449169", "0.0259695556", "-0.00304896012", "-0.00780121796", "-0.0509818681", "-0.00216351566", "-0.0196360387", "-0.00188136974", "-0.0168304425", "-0.0543367341", "-0.0100780325", "-0.120107651", "-0.0243432857", "0.00445867563", "-0.0144569715", "-0.0106938435", "0.0104202135", "-0.0139829824", "0.0142488638", "-0.0521037206", "0.0044897031", "0.0282531139", "0.0185917187", "0.0254540164", "-0.0252824221", "-0.00611533597", "-0.0356582478", "-0.0060582296", "-0.000551598961", "-0.0112998197", "0.0223956686", "0.0310282502", "-0.0261098929", "0.0162973776", "0.0430411696", "0.0175687186", "0.00854843576", "0.0194278434", "-0.0293027181", "-0.024950156", "0.0035575463", "0.00582783855", "-0.0256006233", "-0.00415061414", "-0.00844062", "0.00565143395", "0.0157454368", "-0.00574005209", "0.0100544

In [54]:
with open(summaries_file.name) as f:
    for line in f:
         print(line)

{"id": "7944pcnf.png", "summary": [" The image shows three people: Joe Biden, a young girl, and Hunter Biden. Joe Biden is smiling and wearing a dark suit. The young girl is smiling and wearing a white dress. Hunter Biden is smiling and wearing a dark suit. The background is a photo of the White House."]}



In [67]:
#save the temp files in persistent disk
# import tempfile, shutil
 
# file_name = embeddings_file.name
# embeddings_file.close()
# shutil.copy(file_name, 'embeddings_file.json')
 

'embeddings_file.json'